In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

df = pd.read_csv('/content/nodes.csv')
df.head()

,spotify_id,name,followers,popularity,genres,chart_hits
0,48WvrUGoijadXXCsGocwM4,Byklubben,1738.0,24.0,"['nordic house', 'russelater']",['no (3)']
1,4lDiJcOJ2GLCK6p9q5BgfK,Kontra K,1999676.0,72.0,"['christlicher rap', 'german hip hop']","['at (44)', 'de (111)', 'lu (22)', 'ch (31)', ..."
2,652XIvIBNGg3C0KIGEJWit,Maxim,34596.0,36.0,[],['de (1)']
3,3dXC1YPbnQPsfHPVkm1ipj,Christopher Martin,249233.0,52.0,"['dancehall', 'lovers rock', 'modern reggae', ...","['at (1)', 'de (1)']"
4,74terC9ol9zMo8rfzhSOiG,Jakob Hellman,21193.0,39.0,"['classic swedish pop', 'norrbotten indie', 's...",['se (6)']


In [89]:
#Cleaning:
#Remove whitespace
df.columns = df.columns.str.strip()

#Drop any nas
df = df.dropna(subset = ['spotify_id', 'genres'])

from sklearn.preprocessing import MultiLabelBinarizer

# Parse the 'genres' column into list format
df['genres_parsed'] = df['genres'].apply(lambda x: eval(x) if pd.notnull(x) else [])

# Apply one-hot encoding using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(df['genres_parsed']), columns=mlb.classes_, index=df.index)

# Combine the encoded genres with the original DataFrame
df_encoded = pd.concat([df, genres_encoded], axis=1)

# Drop unnecessary columns for clustering
df_for_clustering = df_encoded.drop(columns=['name', 'genres', 'chart_hits', 'genres_parsed'])
df_for_clustering = df_for_clustering.dropna(subset = ['spotify_id'])

# Display a preview of the transformed dataset
df_for_clustering.head()
df_for_clustering = df_for_clustering.set_index('spotify_id')

# Count the occurrences of each genre (sum of binary columns)
genre_counts = genres_encoded.sum().sort_values(ascending=False)

# Select the top 20 most common genres
top_genres = genre_counts.head(20).index

df_top_genres = pd.concat([df[['spotify_id']], genres_encoded[top_genres]], axis=1)
df_top_genres = df_top_genres.set_index('spotify_id')

<ipython-input-89-678b77fae82a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres_parsed'] = df['genres'].apply(lambda x: eval(x) if pd.notnull(x) else [])


In [90]:
# Combine features
X = pd.concat([df_top_genres, df_for_clustering[['followers']]], axis=1)

# Drop missing values from X
X = X.dropna()

# Define target variable
y = df_for_clustering['popularity']

# Align X and y to common indices
common_indices = X.index.intersection(y.index)
X = X.loc[common_indices]
y = y.loc[common_indices]

# Ensure no missing values
combined = pd.concat([X, y], axis=1).dropna()
X = combined.iloc[:, :-1]
y = combined.iloc[:, -1]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Ridge Regression
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

# Evaluate Model
# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error: 13.515001775718273
Mean Squared Error: 293.92665708962426
R^2 Score: 0.19101685162346616


In [91]:
import pandas as pd

# Create a DataFrame for Actual vs Predicted Popularity
results = pd.DataFrame({
    'Actual Popularity': y_test,
    'Predicted Popularity': y_pred
})

# Add a difference column to compare errors (optional)
results['Difference'] = results['Actual Popularity'] - results['Predicted Popularity']

# Display the first few rows
print(results.head())

                        Actual Popularity  Predicted Popularity  Difference
spotify_id                                                                 
6ZjFtWeHP9XN7FeKSUe80S               55.0             38.245727   16.754273
2eGbmJfjYbZatbebJxY6PD               33.0             37.357791   -4.357791
0E02VcvA5p1ndkLdqWD5JB               68.0             81.762624  -13.762624
0r8toju2ecKaVtItkzAnNi               66.0             60.044993    5.955007
45XwGsD55yVnJFXQN6zUmK               27.0             37.354684  -10.354684
